In [1]:
from joblib import load
import ngram as ng

In [2]:
import os
import pymysql
import pymysql.cursors

In [ ]:
host = os.getenv('localhost')
port = os.getenv('3306')
user = os.getenv('root')
password = os.getenv('')
database = os.getenv('test3')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd=password,
    db="test3",
    charset='utf8mb4')
cursor=conn.cursor()

In [4]:
# load the model from disk TF-idf
filename = 'model/tfdf_model_n_gram.joblib'
tfidf = load(filename)

# load the model from disk SVM
filename = 'model/svm_model_n_gram_irham.joblib'
SVM = load(filename)

In [5]:
proses = "lebih baik lagi"
proses = ng.ngram(proses)
tfidf_matrix_ng = tfidf.transform([proses])

prediksi_lx = SVM.predict(tfidf_matrix_ng.toarray())[0]
print("  "+proses)
print("<< N-Gram  : "+prediksi_lx+">>")

  lebih_baik lagi
<< N-Gram  : Positive>>


In [6]:
def prediksi(proses):
    proses = ng.ngram(proses)
    tfidf_matrix_ng = tfidf.transform([proses])
    prediksi_lx = SVM.predict(tfidf_matrix_ng.toarray())[0]
    return prediksi_lx

prediksi(proses)

'Positive'

In [ ]:
df = pd.read_sql_query("SELECT * FROM komentar_siswa_data_bersih;",conn)
df.head(10)
n_data = list()
id_ = df['id'].tolist()
asli = df['catatan_kuesioner'].tolist() #catatan_kuesioner harus diganti dengan data kotor/data belum masuk preprocessing
for i in df['catatan_kuesioner'].tolist():
    n_data.append(i.replace("[","").replace("]","").replace('"',"").replace(','," "))
n_data

In [ ]:
pred_list = list()
for p, ix in zip(n_data, id_):
    hasil = prediksi(p)
    pred_list.append(hasil)
#     print(hasil)
    sql = "UPDATE komentar_siswa_data_bersih SET class_prediksi = '%s' WHERE `id` = '%s'" % (hasil, ix)
    print(hasil, ix)
    try:
        cursor.execute(sql)
        conn.commit()
    except:
        conn.rollback()

In [ ]:
dictt={
    'id':id_,
    "label prediksi":pred_list,
    "komentar":n_data,
    "komentar_asli":asli,
}
df=pd.DataFrame.from_dict(dictt)
df.to_excel("hasil_prediksi.xlsx")